- join tables
- remove duplicates
- outliers / missing data
- standarize names -> coordinates to names of places
- ...

deixo aixo aqui per si pot servir despres

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

from pathlib import Path

Geopy for inputing zip code

In [2]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'app_adsdb'

#geolocator = Nominatim(user_agent='app_adsdb/1')
geolocator = Nominatim(timeout=3)

In [3]:

def postcode(df,latitude="Latitude",longitude="Longitude"):
    Lat = np.array(df[[latitude]]).flatten()
    Long = np.array(df[[longitude]]).flatten()

    coordinates = [(Lat[i],Long[i]) for i in range(len(Lat))]#len(Lat)
    location = [geolocator.reverse(coor).raw["address"] for coor in coordinates]

    postcode = []

    for loc in location:
        if "postcode" in loc:
            postcode.append(loc["postcode"])
        else:
            postcode.append("")

    return df.assign(ZIP=list(postcode))
    


Drop columns with 90% < missings or is constant

In [4]:
def drop_columns(df,alpha=0.9, constants=True):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    missings = 0
    n = len(df)
    drop_list = []

    for i in range(len(columns_names)):

        if types_df[i] == "float64" or types_df[i] == "int32":
            missings = df[columns_names[i]].isna().sum()
            
            if constants and min(df[columns_names[i]]) == max(df[columns_names[i]]):
                drop_list.append(columns_names[i])
                continue

        elif types_df[i] == "object":
            counts = dict(df[columns_names[i]].value_counts())
            if "unknow" in counts:
                missings = counts["unknow"]

            if constants and len(counts) == 1:
                drop_list.append(columns_names[i])
                continue

        if missings/n > alpha:
            drop_list.append(columns_names[i])

    return df.drop(drop_list,axis=1)


For numeric missings -> np.nan
for the categorical -> unknown

In [5]:
# numeric -> np.nan
# object -> "unknow"
def standar_missings(df):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    
    for i in range(len(columns_names)):
        
        if types_df[i] == "float64" or types_df[i] == "int32":
            df[columns_names[i]] = df[columns_names[i]].replace(-999, np.nan)

        elif types_df[i] == "object":
            df[columns_names[i]] = df[columns_names[i]].replace(np.nan, "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("xxxxxx", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("nan", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("......", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("NOT AVAILABLE", "unknow")
            

In [1]:
def formatted2trusted():
    
    # list of tables already processed:
    processed_table = []
    with open('logs/processed_data_trusted.txt', "r") as f0:
        for line in f0:
            processed_table.append(line.rstrip())
    
    # process datasets:
    for datasource in Objects:
        id = datasource["id"]

        # conect to DB
        con = duckdb.connect(database="data/formatted/db_{}.db".format(id), read_only=True)

        # take all tables names
        tables = con.execute("SHOW TABLES").fetchall()
        tables_names = [x[0] for x in tables]

        # tables to process:
        tables_not_processed = [table for table in tables_names if not table in processed_table]
        if len(tables_not_processed) == 0:
            continue
        
        # list of all tables in dataframe format
        dataframes = []
        for table in tables_not_processed:
            dataframes.append(con.execute("SELECT * FROM {}".format(table)).fetchdf())

        # close database
        con.close()

        # merge all dataframes
        df = pd.concat(dataframes)
        standar_missings(df)
        df.drop_duplicates()
        df = drop_columns(df)

        my_file = Path("data/trusted/db_{}.db".format(id))
        if my_file.is_file():
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            df_original = con.execute("SELECT * FROM {}".format(id)).fetchdf()
            df = pd.concat([df,df_original])
            df.drop_duplicates()#CREATE OR REPLACE TABLE
            df = drop_columns(df)
            con.execute("CREATE OR REPLACE TABLE {} AS SELECT * FROM df".format(id))
            con.close()
        
        else:
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            con.execute("CREATE TABLE {} AS SELECT * FROM df".format(id))
            con.close()

        if os.path.exists("scripts/trusted_{}.py".format(id)):
            os.system("python3 ./scripts/trusted_{}.py".format(id))

        with open('logs/processed_data_trusted.txt', "a") as f0:
            for table in tables_not_processed:
                f0.write(table + "\n")

    

In [2]:
formatted2trusted()

FileNotFoundError: [Errno 2] No such file or directory: 'logs/processed_data_trusted.txt'

Testin specific scripts

In [ ]:
con = duckdb.connect(database="data/trusted/db_schools.db", read_only=False)
df = con.execute("SELECT * FROM schools;").fetchdf()
con.close()
df[["LEVEL_","TYPE","STATUS"]].head(10)

,LEVEL_,TYPE,STATUS
0,preschool,Type 1,STATUS 1
1,elementary_school,Type 1,STATUS 1
2,elementary_school,Type 1,STATUS 1
3,preschool,Type 1,STATUS 1
4,elementary_school,Type 1,STATUS 1
5,preschool,Type 1,STATUS 1
6,elementary_school,Type 1,STATUS 1
7,unknown,Type 1,STATUS 2
8,elementary_school,Type 1,STATUS 1
9,middle_school,Type 1,STATUS 1


In [ ]:
con = duckdb.connect(database="data/trusted/db_housing.db", read_only=False)
df = con.execute("SELECT * FROM housing;").fetchdf()
con.close()
df[["cats_allowed","dogs_allowed","smoking_allowed","wheelchair_access","comes_furnished"]].head(10)

,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,comes_furnished
0,Yes,Yes,Yes,No,No
1,Yes,Yes,Yes,No,Yes
2,Yes,Yes,Yes,No,No
3,Yes,Yes,Yes,No,No
4,Yes,Yes,Yes,No,No
5,Yes,Yes,Yes,No,No
6,Yes,Yes,Yes,No,No
7,Yes,Yes,Yes,No,No
8,Yes,Yes,No,No,No
9,Yes,Yes,Yes,No,No
